In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
import gc

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, GlobalMaxPooling2D, BatchNormalization
from keras.utils import to_categorical
from keras.regularizers import l2
from tensorflow.keras.models import Sequential 

from tensorflow.keras.preprocessing.image import ImageDataGenerator

import keras as k
from keras import backend as K
from keras.models import Sequential

import cv2
from tqdm import tqdm
from collections import Counter

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score

import plotly.express as px

path = '../input/planets-dataset/'
os.listdir(path)

2024-06-04 22:44:50.235251: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-04 22:44:50.235380: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-04 22:44:50.379982: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


['planet', 'test-jpg-additional']

In [2]:
#Loading the image datasets
train_path = '../kaggle/input/planets-dataset/planet/planet/train_classes.csv'
test_path = '../kaggle/input/planets-dataset/planet/planet/sample_submission.csv'
train_images = '../input/planets-dataset/planet/planet/train-jpg'
test_images = ',,/input/planets-dataset/planet/planet/test-jpg'


In [3]:
#train_path = '/kaggle/input/planet-understanding-the-amazon-from-space/train_v2.csv/train_v2.csv'
#test_path = '/kaggle/input/planet-understanding-the-amazon-from-space/test_v2_file_mapping.csv/test_v2_file_mapping.csv'
#submission_path = '/kaggle/input/planet-understanding-the-amazon-from-space/sample_submission_v2.csv/sample_submission_v2.csv'

In [4]:
train_df = pd.read_csv('/kaggle/input/planets-dataset/planet/planet/train_classes.csv')
test_df = pd.read_csv('/kaggle/input/planets-dataset/planet/planet/sample_submission.csv')



In [5]:
train_df.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [6]:
test_df.head()

,image_name,tags
0,test_0,primary clear agriculture road water
1,test_1,primary clear agriculture road water
2,test_2,primary clear agriculture road water
3,test_3,primary clear agriculture road water
4,test_4,primary clear agriculture road water


In [7]:
print (train_df.shape)
print (test_df.shape)


(40479, 2)
(61191, 2)


In [8]:
label_list = []
for tag_split in train_df.tags.to_numpy():
    labels = tag_split.split(' ')
    for label in labels:
        if label not in label_list:
            label_list.append(label)

In [9]:
print(len(label_list))

17


In [10]:
labels_dict = dict(zip(range(0,17), label_list))
labels_dict

{0: 'haze',
 1: 'primary',
 2: 'agriculture',
 3: 'clear',
 4: 'water',
 5: 'habitation',
 6: 'road',
 7: 'cultivation',
 8: 'slash_burn',
 9: 'cloudy',
 10: 'partly_cloudy',
 11: 'conventional_mine',
 12: 'bare_ground',
 13: 'artisinal_mine',
 14: 'blooming',
 15: 'selective_logging',
 16: 'blow_down'}

In [11]:
#  Applying one-hot encoding
for l in label_list:
    train_df[l] = train_df.tags.apply(lambda x: 1 if l in x.split() else 0)
    
train_df.head()

,image_name,tags,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down
0,train_0,haze primary,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,train_1,agriculture clear primary water,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
2,train_2,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,train_3,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,train_4,agriculture clear habitation primary road,0,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0


In [12]:
train_df[label_list].sum().sort_values(ascending=False)

primary              37513
clear                28431
agriculture          12315
road                  8071
water                 7411
partly_cloudy         7261
cultivation           4477
habitation            3660
haze                  2697
cloudy                2089
bare_ground            862
selective_logging      340
artisinal_mine         339
blooming               332
slash_burn             209
conventional_mine      100
blow_down               98
dtype: int64

In [13]:
gc.collect()

60

In [14]:
train_df_columns = list(train_df.columns[2:])
train_df_columns

['haze',
 'primary',
 'agriculture',
 'clear',
 'water',
 'habitation',
 'road',
 'cultivation',
 'slash_burn',
 'cloudy',
 'partly_cloudy',
 'conventional_mine',
 'bare_ground',
 'artisinal_mine',
 'blooming',
 'selective_logging',
 'blow_down']

In [15]:
# onehot encoding the image name
train_df['image_name'] = train_df['image_name'].apply(lambda x: f'{x}.jpg')
train_df.head()

,image_name,tags,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down
0,train_0.jpg,haze primary,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,train_1.jpg,agriculture clear primary water,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
2,train_2.jpg,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,train_3.jpg,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,train_4.jpg,agriculture clear habitation primary road,0,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0


In [16]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split = 0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_dataframe(
    dataframe = train_df,
    directory = train_images,
    x_col = 'image_name',
    y_col = train_df_columns,
    subset = 'training',
    batch_size = 64,
    seed=42,
    shuffle=True,
    class_mode='raw',
    target_size = (150,150))

#validation_datagen = ImageDataGenerator(rescale=1./255)

#validation_generator = validation_datagen.flow_from_dataframe(dataframe = '/kaggle/input/planet-understanding-the-amazon-from-space/test_v2_file_mapping.csv',
    #target_size=(150, 150),
    #batch_size=32,
    #class_mode='categorical'
#)
                                                              
#generating validation data which is expected to be 20% of the train dataset since validation split is 0.2
val_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory =train_images, 
    x_col='image_name',
    y_col= train_df_columns,
    subset='validation', 
    batch_size=32,
    seed=42, 
    shuffle=True, 
    class_mode='raw',
    target_size=(150,150))
                                                              
                                                            

Found 32384 validated image filenames.
Found 8095 validated image filenames.


In [17]:
#setting up step size for training and validation image data
step_train_size = int(np.ceil(train_generator.samples / train_generator.batch_size))
step_val_size = int(np.ceil(val_generator.samples / val_generator.batch_size))
print(step_train_size , step_val_size)

506 253


In [18]:
# Define the CNN model
#model = Sequential([
    #layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    #layers.MaxPooling2D((2, 2)),
    #layers.Conv2D(64, (3, 3), activation='relu'),
    #layers.MaxPooling2D((2, 2)),
    #layers.Conv2D(128, (3, 3), activation='relu'),
    #layers.MaxPooling2D((2, 2)),
    #layers.Flatten(),
    #layers.Dense(512, activation='relu'),
    #layers.Dropout(0.5),
    #layers.Dense (17, activation='sigmoid')
    
#])

# Compile the model
#model.compile(optimizer='adam',
              #loss='binary_crossentropy',
              #metrics=['accuracy'])

# Model architecture
#model = Sequential([
    #Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    #MaxPooling2D(2, 2),
    #Conv2D(64, (3, 3), activation='relu'),
    #MaxPooling2D(2, 2),
    #Conv2D(128, (3, 3), activation='relu'),
    #MaxPooling2D(2, 2),
    #Flatten(),
    #Dense(512, activation='relu'),
    #Dropout(0.5),
    #Dense(num_classes, activation='sigmoid')
#])

#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#history = model.fit(train_generator, epochs=50, validation_data=val_generator)

# Define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy','FBetaScore','CategoricalAccuracy'])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [19]:
history = model.fit(
    x=train_generator,
    validation_data = val_generator,
    steps_per_epochs=step_train_size,
    epochs=10,
    verbose=1
)

TypeError: TensorFlowTrainer.fit() got an unexpected keyword argument 'steps_per_epochs'

In [ ]:
# Model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(17, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator, epochs=10, validation_data=val_generator)